In [1]:
from google.cloud import storage
import os
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "/Users/weiqiang/.dotbot/cloud/quakeflow_wayne.json"

def download_blob(bucket_name, source_blob_name, destination_file_name):

    storage_client = storage.Client()

    bucket = storage_client.bucket(bucket_name)

    blob = bucket.blob(source_blob_name)
    blob.download_to_filename(destination_file_name)

    print("Blob {} downloaded to {}.".format(source_blob_name, destination_file_name))

In [3]:
download_blob("quakeflow", "catalogs/combined_phases.csv", "combined_phases.csv")

Blob catalogs/combined_phases.csv downloaded to combined_phases.csv.


In [4]:
import pandas as pd
import h5py

In [5]:
catalogs = pd.read_csv("combined_phases.csv", sep="\t", dtype=str)

In [12]:
select_catalog = catalogs.sample(100, random_state=123)
select_catalog.to_csv("selected_phases.csv", sep="\t")

In [13]:
if not os.path.exists("data"):
    os.mkdir("data")
for fname in select_catalog["fname"]:
    download_blob("quakeflow", f"data/{fname}", f"data/{fname}")

Blob data/NN.LHV..EH.0384064.npz downloaded to data/NN.LHV..EH.0384064.npz.
Blob data/NC.MCV..EH.0361339.npz downloaded to data/NC.MCV..EH.0361339.npz.
Blob data/NC.JSJ..HN.0352874.npz downloaded to data/NC.JSJ..HN.0352874.npz.
Blob data/NC.MDH1..DP.0276949.npz downloaded to data/NC.MDH1..DP.0276949.npz.
Blob data/BG.NEG..DP.0343655.npz downloaded to data/BG.NEG..DP.0343655.npz.
Blob data/NC.C029.01.HN.0335084.npz downloaded to data/NC.C029.01.HN.0335084.npz.
Blob data/NC.GAX..EH.0169422.npz downloaded to data/NC.GAX..EH.0169422.npz.
Blob data/BK.DANT.00.BH.0373267.npz downloaded to data/BK.DANT.00.BH.0373267.npz.
Blob data/PB.B045..EH.0385872.npz downloaded to data/PB.B045..EH.0385872.npz.
Blob data/BG.ACR..DP.0346833.npz downloaded to data/BG.ACR..DP.0346833.npz.
Blob data/BG.SQK..DP.0399575.npz downloaded to data/BG.SQK..DP.0399575.npz.
Blob data/NC.MDY..HN.0376095.npz downloaded to data/NC.MDY..HN.0376095.npz.
Blob data/NC.MDH1..DP.0412729.npz downloaded to data/NC.MDH1..DP.0412729

In [14]:
select_catalog.to_hdf('data.h5', '/catalog', mode='w', format='table', data_columns=True)
with h5py.File("data.h5", "a", libver='latest') as fp:
#     catalog = fp.create_group("/catalog")
    data = fp.create_group("/data")
    for fname in select_catalog["fname"]:
        meta = np.load(f"data/{fname}")
        ds = data.create_dataset(fname, data=meta["data"], dtype="float32")
        for k in meta:
            if k != "data":
                if meta[k].dtype.type is np.str_:
                    ds.attrs[k] = str(meta[k])
                else:
                    ds.attrs[k] = meta[k]

In [10]:
for fname in select_catalog["fname"]:
    meta = np.load(f"data/{fname}")
    for k in meta:
        print(k, meta[k], meta[k].dtype,len(meta[k]), hasattr(meta[k], "__len__"))
    break

data [[ 0.0000000e+00  0.0000000e+00  4.3978863e-23]
 [ 0.0000000e+00  0.0000000e+00 -4.0226005e-24]
 [ 0.0000000e+00  0.0000000e+00 -5.8025358e-23]
 ...
 [ 0.0000000e+00  0.0000000e+00  3.0300390e-23]
 [ 0.0000000e+00  0.0000000e+00  1.2032863e-22]
 [ 0.0000000e+00  0.0000000e+00  2.4316828e-23]] float32 12000 True


TypeError: len() of unsized object

In [11]:
with h5py.File("data.h5", "r", libver='latest', swmr=True) as fp:
    print(list(fp['data'].keys()))
    for k in fp['data'].keys():
        print(fp[f"data/{k}"].attrs["snr"])

['BG.ACR..DP.0346833.npz', 'BG.NEG..DP.0343655.npz', 'BK.DANT.00.BH.0373267.npz', 'NC.C029.01.HN.0335084.npz', 'NC.GAX..EH.0169422.npz', 'NC.JSJ..HN.0352874.npz', 'NC.MCV..EH.0361339.npz', 'NC.MDH1..DP.0276949.npz', 'NN.LHV..EH.0384064.npz', 'PB.B045..EH.0385872.npz']
[4.55772313 4.40383979 1.08694338]
[2.65475264 3.52927338 2.58486951]
[1.03147117 1.01812326 0.90910357]
[19.95741355 21.85599121 13.06807128]
[1.         1.         1.78825248]
[10.37468862 15.63222592 17.56807564]
[1.         1.         3.21769929]
[1.01823104 1.02137857 1.02093327]
[1. 1. 1.]
[5.97717719 3.32311816 5.04617698]


In [ ]:
pd.read_hdf("data.h5", "/catalog")